In [81]:
#profitableHospital-project

In [82]:
#dependencies
import pandas as pd
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import tensorflow
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import joblib

In [83]:
# ... File Imports

In [84]:
#2017 Hospital Data from Centers for Medicare Services
df = pd.read_csv("Medicare_Hospital_Cost_Report_PUF_2016.csv")
df.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num
0,310050,SAINT CLARES HOSPITAL- DENVILLE,25 POCONO ROAD,DENVILLE,NJ,07834,MORRIS,35644.0,U,STH,...,NaN,8399912.0,NaN,8399912.0,0.226854,2010908.0,14023081.0,NaN,NaN,577882
1,670110,FIRST TEXAS HOSPITAL CARROLLTON,1407 E TRINITY MILLS ROAD,CARROLLTON,TX,75006,NaN,NaN,NaN,STH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,581059


In [85]:
#Quick Check
df['Provider CCN'].count()

6211

In [86]:
#Certificate of Need Status by State File
conTable = pd.read_csv("certificateofneedstatesTable.csv")
conTable.head(2)

,State Code,CON
0,AK,1
1,AL,1


In [87]:
#String to Numeric Conversion Table for Urban v. Rural designation
urbanTable = pd.read_csv("ruralversusurbanTable.csv")
urbanTable.head()

,Rural Versus Urban,Urban
0,U,1
1,R,0


In [88]:
#Lattitude & Longitude for the 2017 hospital listing
geoTable = pd.read_csv("geocoded2017.csv")
geoTable.head(2)

,Provider CCN,lat,lng
0,510013,39.946236,-80.752966
1,340106,34.901612,-79.708561


In [89]:
# ... Appending of source files into one main working file

In [90]:
mainFile1 = df

In [91]:
#Append a lattitude & longitude
#mainFile1 = pd.merge(mainFile1, geoTable, how= "left", on=['Provider CCN'])
#mainFile1.head(2)

In [92]:
#Quick Check
mainFile1['Provider CCN'].count()

6211

In [93]:
#Append certificate of need indicator
mainFile1 = pd.merge(mainFile1, conTable, how= "left", on=['State Code'])
#mainFile1.head()

In [94]:
#Quick Check
mainFile1['Provider CCN'].count()

6211

In [95]:
#Append a numeric representation of rural v. urban
mainFile1 = pd.merge(mainFile1, urbanTable, how= "left", on=['Rural Versus Urban'])

In [96]:
#Quick Check
mainFile1['Provider CCN'].count()

6211

In [97]:
mainFile1.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,CON,Urban
0,310050,SAINT CLARES HOSPITAL- DENVILLE,25 POCONO ROAD,DENVILLE,NJ,07834,MORRIS,35644.0,U,STH,...,NaN,8399912.0,0.226854,2010908.0,14023081.0,NaN,NaN,577882,1.0,1.0
1,670110,FIRST TEXAS HOSPITAL CARROLLTON,1407 E TRINITY MILLS ROAD,CARROLLTON,TX,75006,NaN,NaN,NaN,STH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,581059,0.0,NaN


In [98]:
mainFile1.to_csv("main1_2016.csv", index=False, header=True)

In [99]:
# ... Add calculated fields

In [100]:
mainFile2 = mainFile1

In [101]:
#labor Expense
lbrexpense = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
mainFile2["Labor Exp"] = lbrexpense
#wagepercent.head()

In [102]:
#Append calculated field inpatient percent
inpatient = mainFile2["Inpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Inpatient %"] = inpatient
#mainFile2.head()

In [103]:
#Append calculated field outpatient percent
outpatient = mainFile2["Outpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Outpatient %"] = outpatient
#mainFile2.head()

In [104]:
#Append calculated field charity percent
charity = mainFile2["Cost of Charity Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Charity %"] = charity
#mainFile2.head()

In [105]:
#Append calculated field uncompensated percent
uncompensated = mainFile2["Cost of Uncompensated Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Uncompensated %"] = charity
#mainFile2.head()

In [106]:
#Append calculated field revenue per discharge
revdischarge = mainFile2["Net Patient Revenue"]/mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]
mainFile2["Rev/Discharge"] = revdischarge
#mainFile2.head()

In [107]:
#Append calculated field revenue per bed
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Number of Beds"]
mainFile2["Rev Per Bed"] = revbed
#mainFile2.head()

In [108]:
#Append calculated field revenue per bed day
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Total Bed Days Available"]
mainFile2["Rev Per Bed Days"] = revbed
#mainFile2.head()

In [109]:
#Append calculated field revs per fte
# revfte = mainFile2["Net Patient Revenue"]/mainFile2["FTE - Employees on Payroll"]
# mainFile2["Rev Per FTE"] = revfte
#mainFile2.head()

In [110]:
#Append calculated field discharges per bed
volbed = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Number of Beds"]
mainFile2["Discharges Per Bed"] = volbed
#mainFile2.head()

In [111]:
#Append calculated field discharges per bed day
voldays = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Total Bed Days Available"]
mainFile2["Discharges Per Bed Days"] = voldays
#mainFile2.head()

In [112]:
#Append calculated field discharges per bed
volfte = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Discharge/FTE"] = volbed
#mainFile2.head()

In [113]:
#labor percent
wagepercent = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagepercent = wagepercent / mainFile2["Net Patient Revenue"]
mainFile2["Labor %"] = wagepercent
#wagepercent.head()

In [114]:
#Overhead percent
overheadpercent = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Net Patient Revenue"]
mainFile2["Overhead %"] = overheadpercent
#overheadpercent.head()

In [115]:
#Depreciation percent
deprpercent = mainFile2["Depreciation Cost"] / mainFile2["Net Patient Revenue"]
mainFile2["Depreciation %"] = deprpercent
#overheadpercent.head()

In [116]:
#labor Per FTE
wagefte = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagefte = wagefte / mainFile2["FTE - Employees on Payroll"]
mainFile2["LbrExp/FTE"] = wagefte
#wagepercent.head()

In [117]:
#Overhead Expense Per Bed
overbed = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Number of Beds"]
mainFile2["Overhead Exp Per Bed"] = overbed
#wagepercent.head()

In [118]:
#Append calculated field net income percent to file
profit = mainFile2["Net Income"]/mainFile2["Net Patient Revenue"]
mainFile2["Net Income %"] = profit
#mainFile2.head()

In [119]:
# Review net income percent for outlier reduction approach
netinc = mainFile2["Net Income %"]
#netinc.head()

In [120]:
# Box plot and outlier calculations for outlier approach
# netinc2 = netinc.dropna(how='any')
# chartdata = netinc2
# fig1, ax1 = plt.subplots()
# ax1.set_title('Net Income Percent')
# ax1.set_ylabel('Net Income Percent')
# ax1.boxplot(chartdata)
# plt.show()

In [121]:
#Quartile calculations
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile of profit percent is: {lowerq}")
# print(f"The upper quartile of profit percent is: {upperq}")
# print(f"The interquartile range of profit percent is: {iqr}")
# print(f"The the median of profit percent is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [122]:
#Create a bronze, silver, gold score for clasifying net income performance
bins = [-100, -0.030605, 0.113797, 100]
group_names = ["3", "2", "1"]

In [123]:
#Quintile attempt
# quartiles = chartdata.quantile([.2,.5,.8])
# lowerq = quartiles[0.2]
# upperq = quartiles[0.8]
# iqr = upperq-lowerq

# print(f"The lower quartile of profit percent is: {lowerq}")
# print(f"The upper quartile of profit percent is: {upperq}")
# print(f"The interquartile range of profit percent is: {iqr}")
# print(f"The the median of profit percent is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [124]:
#Create a bronze, silver, gold score for clasifying net income performance
# bins = [-100, -0.0542, .03804, 0.13654, 100]
# group_names = ["4","3", "2", "1"]

In [125]:
#Add a bronze, silver, gold scoring system to the dataset
mainFile2["Net Income Score"] = pd.cut(mainFile2["Net Income %"], bins, labels=group_names, include_lowest=True)
#mainFile2.head()

In [126]:
#Quick check on distribution of bronze, silver and gold
mainFile2['Net Income Score'].value_counts()

2    3036
1    1458
3    1446
Name: Net Income Score, dtype: int64

In [127]:
#Quick Check
mainFile2['Provider CCN'].count()

6211

In [128]:
mainFile2.to_csv("main2_2016.csv", index=False, header=True)

In [129]:
# ..... File targeting

In [130]:
mainFile3 = mainFile2
mainFile3['Provider CCN'].count()

6211

In [131]:
#Targets dataset on Colorad0
#mainFile3 = mainFile3.loc[df['State Code'] == 'CO']

In [132]:
#Removes state
mainFile3 = mainFile3.loc[df['State Code'] != 'PR']

In [133]:
#Removes state
mainFile3 = mainFile3.loc[df['State Code'] != 'GU']

In [134]:
#Removes state
mainFile3 = mainFile3.loc[df['State Code'] != 'VI']

In [135]:
#Removes state
mainFile3 = mainFile3.loc[df['State Code'] != 'MP']

In [136]:
#Removes state
mainFile3 = mainFile3.loc[df['State Code'] != 'HI']

In [137]:
#Removes state
mainFile3 = mainFile3.loc[df['State Code'] != 'AK']

In [138]:
#Quick Check ... Lose 117 to state code reduction
mainFile3['Provider CCN'].count()

6095

In [139]:
#Remove hospitals that did not report revenue (Lose 273 to missing or unreported revenue events)
mainFile3 = mainFile3.loc[mainFile2['Net Patient Revenue'] >= 0.1]

In [140]:
#Quick Check
mainFile3['Provider CCN'].count()

5840

In [141]:
#Remove hospitals that discharged less than 15 patients (Lose 49 by placing a 15 patient minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile2['Total Discharges (V + XVIII + XIX + Unknown)'] >= 15]

In [142]:
#Quick Check
mainFile3['Provider CCN'].count()

5793

In [143]:
#Remove hospitals with less than 10 employees on payroll (Lose 23 by placing a 10 employee minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile2['FTE - Employees on Payroll'] >= 10]

In [144]:
#Quick Check
mainFile3['Provider CCN'].count()

5759

In [145]:
#Targets dataset on certificate of need states
#mainFile3 = mainFile3.loc[mainFile2['CON'] == 1]

In [146]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] <= .3304]

In [147]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] >= -.2473]

In [148]:
#Remove Discharges Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 90000]

In [149]:
#Remove Rev Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Bed'] <= 2000000]

In [150]:
#Remove Rev Per FTE outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per FTE'] <= 500000]

In [151]:
#Remove Rev Per Discharge outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 175000]

In [152]:
#Targets dataset on STH locations (~56% of eligible continental US locations are STH providers)
mainFile3 = mainFile3.loc[mainFile2['CCN Facility Type'] == 'STH']

In [153]:
#Quick Check
mainFile3['Provider CCN'].count()

3236

In [154]:
#Quick check on distribution of bronze, silver and gold
#mainFile3['Net Income Score'].value_counts()

In [155]:
#Quick Check
mainFile3['Provider CCN'].count()

3236

In [156]:
mainFile3.to_csv("main3_2016.csv", index=False, header=True)

In [157]:
# ... Target file based on relevant features

In [158]:
mainFile4 = mainFile3

In [159]:
#Targeting the dataset on metrics important to Kent
mainFile4 = mainFile4[['Provider CCN','Hospital Name','Street Address','City','State Code','Zip Code', 'Rural Versus Urban', 'CCN Facility Type', 'Fiscal Year Begin Date', 'Fiscal Year End Date', 'FTE - Employees on Payroll', 'Number of Beds', 'Total Bed Days Available', 'Total Discharges (V + XVIII + XIX + Unknown)', 'Net Patient Revenue', 'Net Income', 'Wage-Related Costs (Core)', 'Total Salaries (adjusted)', 'CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE','Net Income %', 'Net Income Score', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed']]
mainFile4.head(2)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,Rural Versus Urban,CCN Facility Type,Fiscal Year Begin Date,Fiscal Year End Date,...,Discharges Per Bed,Discharges Per Bed Days,Discharge/FTE,Net Income %,Net Income Score,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed
0,310050,SAINT CLARES HOSPITAL- DENVILLE,25 POCONO ROAD,DENVILLE,NJ,07834,U,STH,10/01/2015,12/31/2015,...,12.495385,0.135819,12.495385,0.105576,2,0.436872,0.541184,0.034317,19215.838152,132486.553846
2,340119,STANLY REGIONAL MEDICAL CENTER,301 YADKIN STREET,ALBEMARLE,NC,28001,U,STH,01/01/2016,02/29/2016,...,6.463918,0.107732,6.463918,-0.103473,3,0.594935,0.708396,0.107413,31006.724429,90054.536082


In [160]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    3236
Hospital Name                                   3236
Street Address                                  3235
City                                            3236
State Code                                      3236
Zip Code                                        3236
Rural Versus Urban                              3236
CCN Facility Type                               3236
Fiscal Year Begin Date                          3236
Fiscal Year End Date                            3236
FTE - Employees on Payroll                      3236
Number of Beds                                  3236
Total Bed Days Available                        3236
Total Discharges (V + XVIII + XIX + Unknown)    3236
Net Patient Revenue                             3236
Net Income                                      3235
Wage-Related Costs (Core)                       3226
Total Salaries (adjusted)                       3227
CON                                           

In [161]:
mainFile4 = mainFile4.dropna(how='any')

In [162]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    2997
Hospital Name                                   2997
Street Address                                  2997
City                                            2997
State Code                                      2997
Zip Code                                        2997
Rural Versus Urban                              2997
CCN Facility Type                               2997
Fiscal Year Begin Date                          2997
Fiscal Year End Date                            2997
FTE - Employees on Payroll                      2997
Number of Beds                                  2997
Total Bed Days Available                        2997
Total Discharges (V + XVIII + XIX + Unknown)    2997
Net Patient Revenue                             2997
Net Income                                      2997
Wage-Related Costs (Core)                       2997
Total Salaries (adjusted)                       2997
CON                                           

In [163]:
#Output targeted file
mainFile4.to_csv("main4_2016.csv", index=False, header=True)

In [164]:
# ... Net Income to feature review

In [165]:
targetFile5 = mainFile4

In [166]:
#Outlier Review
chartdata = targetFile5["Discharges Per Bed"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 32.62
The upper quartile of profit percent is: 54.707692307692305
The interquartile range of profit percent is: 22.087692307692308
The the median of profit percent is: 45.542857142857144 
Values below -0.5115384615384642 could be outliers.
Values above 87.83923076923077 could be outliers.


In [167]:
#NI v. discharges per bed
# x_axis = targetFile5["Discharges Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed")
# plt.xlabel("Discharges Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [168]:
#NI v. discharges per bed days
# x_axis = targetFile5["Discharges Per Bed Days"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed Days")
# plt.xlabel("Discharges Per Bed Days")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# #ax.set_yscale('log')
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [169]:
#Outlier Review
chartdata = targetFile5["Rev Per Bed"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 787712.6603773584
The upper quartile of profit percent is: 1518608.4897959183
The interquartile range of profit percent is: 730895.8294185599
The the median of profit percent is: 1121101.310810811 
Values below -308631.08375048137 could be outliers.
Values above 2614952.2339237584 could be outliers.


In [170]:
#NI v. Revenue Per Bed
# x_axis = targetFile5["Rev Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Bed")
# plt.xlabel("Rev Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [171]:
#Outlier Review
chartdata = targetFile5["Rev Per FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

KeyError: 'Rev Per FTE'

In [172]:
#NI v. Rev Per FTE
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Rev Per FTE'] <= 1000000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Rev Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Rev Per FTE")
# plt.xlabel("Rev Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [173]:
#Outlier Review
chartdata = targetFile5["Rev Per Discharge"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

KeyError: 'Rev Per Discharge'

In [174]:
#NI v. Rev Per Discharge
# x_axis = targetFile5["Rev Per Discharge"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Discharge")
# plt.xlabel("Rev Per Discharge")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [175]:
#Outlier Review
chartdata = targetFile5["Labor Exp Per FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

KeyError: 'Labor Exp Per FTE'

In [176]:
#NI v. Labor Expense Per FTE
# targetFile6 = targetFile5.loc[targetFile5['Labor Exp Per FTE'] <= 130000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor Exp Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor Exp Per FTE")
# plt.xlabel("Labor Exp Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [177]:
#Outlier Review
chartdata = targetFile5["Overhead %"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 0.5601878468367469
The upper quartile is: 0.6835785980839738
The interquartile range is: 0.12339075124722687
The the median of is: 0.6156704774180211 
Values below 0.37510171996590663 could be outliers.
Values above 0.8686647249548141 could be outliers.


In [178]:
#NI v. Overhead Percent
# targetFile6 = targetFile5.loc[targetFile5['Overhead %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Overhead %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Overhead %")
# plt.xlabel("Overhead %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [179]:
#Outlier Review
chartdata = targetFile5["Labor %"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 0.3761583013038209
The upper quartile is: 0.53148780615695
The interquartile range is: 0.15532950485312913
The the median of is: 0.44938383983106195 
Values below 0.1431640440241272 could be outliers.
Values above 0.7644820634366437 could be outliers.


In [180]:
#NI v. Labor Percent
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Labor %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor %")
# plt.xlabel("Labor %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [181]:
#Outlier Review
chartdata = targetFile5["Discharges Per FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

KeyError: 'Discharges Per FTE'

In [182]:
#NI v. Labor Percent
# targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Discharges Per FTE'] <= 120]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Discharges Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Discharges Per FTE")
# plt.xlabel("Discharges Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [183]:
#Start machine learning ...

In [184]:
# Select features to be used as x values
features1 = mainFile4[['CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed']]
features1.head(2)
#RFE logrithmic output ... array([15, 17, 16, 14, 12,  8, 11,  6,  5 (R/BDay),  7,  9,  3(D/FTE),  2(L%),  1 (OH%),  4 (D%), 13, 10])

,CON,Urban,Inpatient %,Outpatient %,Charity %,Uncompensated %,Rev/Discharge,Rev Per Bed,Rev Per Bed Days,Discharges Per Bed,Discharges Per Bed Days,Discharge/FTE,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed
0,1.0,1.0,0.609156,0.390844,0.018000,0.018000,19591.934499,244808.756923,2660.964749,12.495385,0.135819,12.495385,0.436872,0.541184,0.034317,19215.838152,132486.553846
2,1.0,1.0,0.312861,0.687139,0.008871,0.008871,19666.786284,127124.484536,2118.741409,6.463918,0.107732,6.463918,0.594935,0.708396,0.107413,31006.724429,90054.536082


In [185]:
#Output features file as a reference document
features1.to_csv("features1_2016.csv", index=False, header=True)

In [188]:
# #Create a training and testing dataset - one hot
# X_train, X_test, y_train, y_test = train_test_split(
#     XX, yy, random_state=1)

In [189]:
y_train.shape

NameError: name 'y_train' is not defined

In [190]:
X_train.shape

NameError: name 'X_train' is not defined

In [191]:
y_test.shape

NameError: name 'y_test' is not defined

In [192]:
y_test.shape

NameError: name 'y_test' is not defined

In [193]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

NameError: name 'X_train' is not defined

In [194]:
#Create the logistic regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LogisticRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

NameError: name 'X_train_scaled' is not defined

In [195]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

NameError: name 'X_test_scaled' is not defined

In [196]:
# ... recursive feature elimination

In [197]:
#Identify features to create a more efficient model with
estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X_train_scaled, y_train)

NameError: name 'X_train_scaled' is not defined

In [198]:
selector.support_

AttributeError: 'RFE' object has no attribute 'support_'

In [199]:
selector.ranking_

AttributeError: 'RFE' object has no attribute 'ranking_'

In [200]:
#End ... kent matthew rodgers, profitableHospital-project